In [14]:
import json
import os
import sys
import math
import datetime
import pathlib

dir_path = pathlib.Path(os.path.abspath('')).resolve()
print(dir_path)
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

/mnt/OpenFormat/python/encoding_size


In [15]:
num_rows = 1000 * 1000
num_cols = 20
sort_list = [
                0.01,
                0.2,
                0.4,
                0.6,
                0.8,
                1.0,
                1.2,
                1.4,
                1.6,
                1.8,
                2.0
            ]
type_list = ['int', 'float', 'string']

In [16]:
%%time
# generate csv files
os.chdir(f'{PROJ_SRC_DIR}/benchmark/generator_v2')
!pwd
for t in type_list:
    print(t)
    for s in sort_list:
        print(s)
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        os.system(f'python3 gen_workloads.py \
            {t} {num_rows} {num_cols} {base_name} zipf {s}')
        os.system(f'cp -r {base_name} {dir_path} && \
            rm -r {base_name}')
os.chdir(dir_path)
!pwd


/mnt/OpenFormat/benchmark/generator_v2
int
0.01
🤔 INFO: too many collisions in int_unique_list, 6418 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 31870 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 8482 values just use (max_value + 1)!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
0.2
🤔 INFO: too many collisions in int_unique_list, 6963 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 33096 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 8482 values just use (max_value + 1)!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
0.4
🤔 INFO: too many collisions in int_unique_list, 7716 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 34131 values just use (max_value + 1)!
🤔 INFO: too many collisions in int_unique_list, 8482 values just use (max_value + 1)!
Finish core dataset! 🎉🎉🎉🎉🎉🎉
Generating Done!!
0.6
🤔 INFO: too many collisio

In [17]:
pq_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/pq_encoding.json')
orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_encoding.json')
pq_name_suffix = '_' + "_".join([str(i) for i in list(pq_config[0].values())]) + '.parquet'
orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
print(pq_config)

loading config from /mnt/OpenFormat/python/experiments/pq_encoding.json ...
loading config from /mnt/OpenFormat/python/experiments/orc_encoding.json ...
[{'compression': 'NONE', 'use_dictionary': True, 'dictionary_pagesize_limit': 2147483647, 'version': '2.6', 'data_page_version': '2.0'}]


In [18]:
# generate parquet and orc files
os.chdir(f'{PROJ_SRC_DIR}/python')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        os.system(f'cp {dir_path}/{base_name}/gen_data/{base_name}.csv data_gen')
        os.system(f'python3 scripts/parquet_exp.py {base_name} pq_encoding -r=false -o=false')
        os.system(f'python3 scripts/orc_exp.py {base_name} orc_encoding -r=false -o=false')
        # orc_schema = open(f'encoding/{base_name}/gen_data/{base_name}_orc_schema.txt', 'r').read()
        # os.system(f'{HOME_DIR}/orc/build/tools/src/csv-import \"{orc_schema}\" \
        #         encoding/{base_name}/gen_data/{base_name}.csv {base_name}.orc')
        os.system(f'mv {base_name}{pq_name_suffix} {dir_path}/{base_name}.parquet')
        os.system(f'mv {base_name}{orc_name_suffix} {dir_path}/{base_name}.orc')
        # os.system(f'mv {base_name}.orc encoding/{base_name}.orc')
os.chdir(dir_path)

loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.4129910469055176
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 0.7526178359985352
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.4052574634552002
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 0.7488794326782227
loading config from ./experiments/pq_encoding.json ...
----Running experiment: 1 out of 1----
NONE_True_2147483647_2.6_2.0
write parquet time (s): 0.38497471809387207
loading config from ./experiments/orc_encoding.json ...
----Running experiment:----
UNCOMPRESSED_speed_1_2147486647
write orc time (s): 0.7478790283203125
loading config from ./experiments/pq

In [19]:
# collect size
os.chdir(dir_path)
os.system('rm outputs/stats.json')
for t in type_list:
    for s in sort_list:
        base_name = f'{t}_r{num_rows}_c{num_cols}_zipf{s}'
        output_stats = {}
        output_stats['type'] = t
        output_stats['feature'] = 'zipf'
        output_stats['param'] = s
        output_stats['file'] = 'parquet'
        output_stats['size'] = os.path.getsize(f'{base_name}.parquet')
        parse_output(output_stats)
        output_stats['file'] = 'orc'
        output_stats['size'] = os.path.getsize(f'{base_name}.orc')
        parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('encoding_size_zipf', timestamp))

0